# Receipt OCR  

### ressources

http://asprise.com/ocr/api/docs/html/receipt-ocr.html <br/>
https://asprise.com/receipt-ocr-data-capture-api/extract-text-reader-scanner-index.html

In [35]:
import os
import requests
import pandas as pd
import json
from googletrans import Translator


print("=== Python Receipt OCR using API")

=== Python Receipt OCR using API


### OCRing the receipt

In [95]:
OverAsked = True
receiptOcrEndpoint = 'https://ocr.asprise.com/api/v1/receipt' # Receipt OCR API endpoint
folder_path_img = "receipts/jpg" #folder path
folder_path_json = "receipts/json" 
folder_path_pdf = "receipts/pdf"

### Convert pdf to jpg

In [98]:
from pdf2image import convert_from_path
 
 
# Store Pdf with convert_from_path function
images = convert_from_path(folder_path_pdf)
   
for i in range(len(images)): 
    # Save pages as images in the pdf
    images[i].save(folder_path_img + 'page'+ str(i) +'.jpg', 'JPEG')




ModuleNotFoundError: No module named 'pdf2image'

#### Read jpg file and save into json

In [96]:
#iterate over the files in the folder 
if OverAsked == False:
  for filename in os.listdir(folder_path_img):
    if filename.endswith(".jpg"):
        image_path = os.path.join(folder_path_img, filename)
        r = requests.post(receiptOcrEndpoint, data = { \
        'client_id': 'TEST',        # Use 'TEST' for testing purpose \
        'recognizer': 'auto',       # can be 'US', 'CA', 'JP', 'SG' or 'auto' \
        'ref_no': 'ocr_python_123', # optional caller provided ref code \
        }, \
        files = {"file": open(image_path, "rb")})

        # Save the json response to a file
        json_filename = os.path.splitext(filename)[0] + ".json"
        json_path = os.path.join(folder_path_json, json_filename)        
        with open(json_path, "w") as outfile:
            json.dump(r.json(), outfile)
            #https://www.geeksforgeeks.org/reading-and-writing-json-to-a-file-in-python/

        print(f"Processed {filename} - Result saved to {json_filename}")
        print(r.text)

In [63]:
with open(folder_path_json+"/test.json", "r") as openfile:
    #reading from json file
    json_object = json.load(openfile)

print(json_object)

{'ocr_type': 'receipts', 'request_id': 'P_92.184.97.11_lj1jfdr2_i4p', 'ref_no': 'AspDemo_1687099400190_538', 'file_name': 'test.jpg', 'request_received_on': 1687099406943, 'success': True, 'image_width': 694, 'image_height': 1650, 'image_rotation': 0.009, 'recognition_completed_on': 1687099408070, 'receipts': [{'merchant_name': 'LIDL Müllerstraße 48-48a', 'merchant_address': '13349 Berlin-Wedding', 'merchant_phone': None, 'merchant_website': None, 'merchant_tax_reg_no': None, 'merchant_company_reg_no': None, 'region': None, 'mall': None, 'country': 'DE', 'receipt_no': 'DE813388858', 'date': '2023-06-17', 'time': '14:15', 'items': [{'amount': 1.12, 'category': None, 'description': 'Bananen kg', 'flags': ' A', 'qty': None, 'remarks': None, 'tags': None, 'unitPrice': None}, {'amount': 2.99, 'category': None, 'description': 'Apfel rot, fru.-süß', 'flags': ' A', 'qty': None, 'remarks': None, 'tags': None, 'unitPrice': None}, {'amount': 1.99, 'category': None, 'description': 'Mexikanisches G

### Find all the items in the receipt

In [64]:
if json_object["receipts"] is not None: 
    receipt_object = json_object["receipts"]
    # Access the "items" list within the "receipts" object
    items = json_object["receipts"][0]["items"]
    total = json_object["receipts"][0]["total"]
    taxes = json_object["receipts"][0]["tax"]
    print("items: ",items)
    print("total: ",total)
    print("taxes: ",taxes)

else:
    print("Error is"+json_object["message"])

items:  [{'amount': 1.12, 'category': None, 'description': 'Bananen kg', 'flags': ' A', 'qty': None, 'remarks': None, 'tags': None, 'unitPrice': None}, {'amount': 2.99, 'category': None, 'description': 'Apfel rot, fru.-süß', 'flags': ' A', 'qty': None, 'remarks': None, 'tags': None, 'unitPrice': None}, {'amount': 1.99, 'category': None, 'description': 'Mexikanisches Gemüse', 'flags': ' A', 'qty': None, 'remarks': None, 'tags': None, 'unitPrice': None}, {'amount': 1.99, 'category': None, 'description': 'Feinschmeckergemüse', 'flags': ' A', 'qty': None, 'remarks': None, 'tags': None, 'unitPrice': None}, {'amount': 2.49, 'category': None, 'description': 'Knuspermüsli + Schoko', 'flags': ' A', 'qty': None, 'remarks': None, 'tags': None, 'unitPrice': None}, {'amount': 0.89, 'category': None, 'description': 'Apfelmus gezuckert', 'flags': ' A', 'qty': None, 'remarks': None, 'tags': None, 'unitPrice': None}, {'amount': 0.75, 'category': None, 'description': 'Traube-Nuß Sch. 100g', 'flags': ' A

### Translation

Ressources : https://www.thepythoncode.com/article/translate-text-in-python

In [39]:
#init the Google API translator
translator = Translator()

In [43]:
for item in items: 
    description = item["description"]
    translation = translator.translate(description, dest="fr")
    item["description"] = translation.text
    print("Description : ", item["description"])

Description :  Banane kg
Description :  oignons jaunes
Description :  PC mangue
Description :  Pêche
Description :  Légumes asiatiques
Description :  Bioland Type 630
Description :  H-lait H, 1,5% OGT
Description :  Fr.aier.a.bo.ha octobre
Description :  Sel de piment
Description :  Fusillis
Description :  Sauce de pomme sugared
Description :  Carotte multi-fruits
Description :  Canneberries sult.mix
Description :  Pain maître
Description :  Poulet
Description :  payer
Description :  Carte de crédit
Description :  Un 7% 1 6523,54
Description :  B 19% 0,321,67


### Product

In [ ]:
#iterate over each item and add them if flaf != none
products = []

for item in items:
    flag = item["flags"]
    if flag != "":
        products.append(item)
    #elif :
        #find total price
    else:
        #nothing to do
        print()
print(products)





[{'amount': 1.19, 'category': None, 'description': 'Banane kg', 'flags': ' A', 'qty': None, 'remarks': None, 'tags': None, 'unitPrice': None}, {'amount': 2.99, 'category': None, 'description': 'Nectarine', 'flags': ' A', 'qty': None, 'remarks': None, 'tags': None, 'unitPrice': None}, {'amount': 1.79, 'category': None, 'description': 'Del.Koch Ham', 'flags': ' A', 'qty': None, 'remarks': None, 'tags': None, 'unitPrice': None}, {'amount': 0.89, 'category': None, 'description': 'Sauce de pomme sugared', 'flags': ' A', 'qty': None, 'remarks': None, 'tags': None, 'unitPrice': None}, {'amount': 0.99, 'category': None, 'description': 'Jeunes haricots', 'flags': ' A', 'qty': None, 'remarks': None, 'tags': None, 'unitPrice': None}, {'amount': 4.05, 'category': None, 'description': 'Papier toilette 3lg', 'flags': ' B', 'qty': None, 'remarks': None, 'tags': None, 'unitPrice': None}, {'amount': 2.85, 'category': None, 'description': 'Torchons de cuisine', 'flags': ' B', 'qty': None, 'remarks':

### Pandas dataframe

In [ ]:
items_df = pd.DataFrame(items)
products_df = pd.DataFrame(products)

In [ ]:
export_csv = pd.DataFrame(products_df[["description", "amount"]])
print(export_csv)
export_csv.to_csv('receipts/export_data.csv', index=False, sep=";")

               description  amount
0                Banane kg    1.19
1                Nectarine    2.99
2             Del.Koch Ham    1.79
3   Sauce de pomme sugared    0.89
4          Jeunes haricots    0.99
5      Papier toilette 3lg    4.05
6      Torchons de cuisine    2.85
7                      Par    0.65
8      Hygienespray Univer    1.95
9                   Éponge    0.75
10   Sacs à déchets 601 25    1.89
11              Tout usage    1.99
12   Fresh Hold '' 0384936    9.99


## All in one

In [93]:
#init the Google API translator
translator_all = Translator()
#initialize
merchant_name_all = []
country_all = []
date_all = []
products_all = []
total_all = []
taxes_all = []
items_all_all = []


for filename in os.listdir(folder_path_json):
    #read json file
    if filename.endswith(".json"):
        json_path = os.path.join(folder_path_json, filename)
        #Read all json in json folder
        with open(json_path, "r") as openfile:
            #reading from json file
            json_object_all = json.load(openfile)
        print(json_object_all)

        if json_object_all["success"] != False:
            #Sort the items, ... in the reicept
            if json_object_all["receipts"] is not None:
                receipt_object_all = json_object_all["receipts"]
                #receipt_object_all.append(json_object_all["receipts"])
                # Access the "items" list within the "receipts" object
                items_all = json_object_all["receipts"][0]["items"]
                #items_all.append(json_object_all["receipts"][0]["items"])
                print("items: ",items)
                total_all.append(json_object_all["receipts"][0]["total"])
                print("total: ",total_all)
                taxes_all.append(json_object_all["receipts"][0]["tax"])
                merchant_name_all.append(json_object_all["receipts"][0]["merchant_name"])
                country_all.append(json_object_all["receipts"][0]["country"])
                date_str = str(json_object_all["receipts"][0]["time"]) if json_object_all["receipts"][0]["time"] else ""
                date_str += "-"
                date_str += str(json_object_all["receipts"][0]["date"]) if json_object_all["receipts"][0]["date"] else ""
                date_all.append(date_str)

                #add for each item merchant_name country_all, date
                for item in items_all:
                    item.update({"merchant_name": json_object_all["receipts"][0]["merchant_name"]})
                    item.update({"country": json_object_all["receipts"][0]["country"]})
                    item.update({"date": date_str})  
            else:
                print("Error is"+json_object_all["message"])

            #translate all the descriptions
            """try:
                for item in items_all: 
                    description_all = item["description"]
                    translation_all = translator_all.translate(item["description"], dest="fr")
                    item["description"] = translation_all.text
                    print("Description : ", item["description"])
            except:
                print("translation canceled")"""          

            #find all the product, only the product -> find with tag ?
            for item in items_all:
                flag = item["flags"]
                if flag != "":
                    products_all.append(item)
            print(products_all)

            #store all items
            for item in items_all:
                items_all_all.append(item)
        
#save data into csv file
if items_all_all and products_all is not None:
    #export only product
    pd.DataFrame(pd.DataFrame(products_all)[["description", "amount", "date"]]).to_csv('receipts/export_product_only.csv', index=False, sep=";")
    #export product all
    pd.DataFrame(pd.DataFrame(products_all)).to_csv('receipts/export_all_product.csv', index=False, sep=";")
    #export items
    #pd.DataFrame(pd.DataFrame(items_all_all)).to_csv('receipts/export_all_items.csv', index=False, sep=";")


{'ocr_type': 'receipts', 'request_id': 'P_92.184.97.11_lj1c71nq_ur8', 'ref_no': 'ocr_python_123', 'file_name': '1.jpg', 'request_received_on': 1687087260711, 'success': True, 'image_width': 1117, 'image_height': 1650, 'image_rotation': -0.033, 'recognition_completed_on': 1687087261622, 'receipts': [{'merchant_name': 'Zwiebeln gelb', 'merchant_address': None, 'merchant_phone': None, 'merchant_website': None, 'merchant_tax_reg_no': None, 'merchant_company_reg_no': None, 'region': None, 'mall': None, 'country': 'DE', 'receipt_no': None, 'date': None, 'time': None, 'items': [{'amount': 1.36, 'category': None, 'description': 'Bananen kg', 'flags': ' A', 'qty': None, 'remarks': None, 'tags': None, 'unitPrice': None}, {'amount': 2.99, 'category': None, 'description': 'Zwiebeln gelb', 'flags': ' A', 'qty': None, 'remarks': None, 'tags': None, 'unitPrice': None}, {'amount': 1.39, 'category': None, 'description': 'Mango Stk', 'flags': ' A', 'qty': None, 'remarks': None, 'tags': None, 'unitPrice'

# Analyse all product and CSV file

In [94]:
#import and create dataset
df_products = pd.read_csv("receipts/export_all_product.csv", sep=";")
#df_items = pd.read_csv("receipts/export_all_items.csv", sep=";")

df_products.head()
#df_products.tail(10)

,amount,category,description,flags,qty,remarks,tags,unitPrice,merchant_name,country,date
58,1.15,NaN,Schwammtuch 5er,B,NaN,NaN,NaN,NaN,TomateStrau. kg,DE,-
59,1.35,NaN,Frischhaltefolie,B,NaN,NaN,NaN,NaN,TomateStrau. kg,DE,-
60,1.95,NaN,Gefrierbeutel 11,B,NaN,NaN,NaN,NaN,TomateStrau. kg,DE,-
61,1.12,NaN,Bananen kg,A,NaN,NaN,NaN,NaN,LIDL Müllerstraße 48-48a,DE,14:15-2023-06-17
62,2.99,NaN,"Apfel rot, fru.-süß",A,NaN,NaN,NaN,NaN,LIDL Müllerstraße 48-48a,DE,14:15-2023-06-17
63,1.99,NaN,Mexikanisches Gemüse,A,NaN,NaN,NaN,NaN,LIDL Müllerstraße 48-48a,DE,14:15-2023-06-17
64,1.99,NaN,Feinschmeckergemüse,A,NaN,NaN,NaN,NaN,LIDL Müllerstraße 48-48a,DE,14:15-2023-06-17
65,2.49,NaN,Knuspermüsli + Schoko,A,NaN,NaN,NaN,NaN,LIDL Müllerstraße 48-48a,DE,14:15-2023-06-17
66,0.89,NaN,Apfelmus gezuckert,A,NaN,NaN,NaN,NaN,LIDL Müllerstraße 48-48a,DE,14:15-2023-06-17
67,0.75,NaN,Traube-Nuß Sch. 100g,A,NaN,NaN,NaN,NaN,LIDL Müllerstraße 48-48a,DE,14:15-2023-06-17
